In [5]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization, LeakyReLU
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Flatten, Conv2D, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

LOADING IN DATASET

In [6]:
from PIL import UnidentifiedImageError
def load_data(dir, img_size):
    images = []
    for filename in os.listdir(dir):
        if filename.endswith("jpg") or filename.endswith("png"):
            try:
                img = load_img(os.path.join(dir, filename), target_size=(img_size, img_size))
                img = img_to_array(img)
                img = (img - 127.5) / 127.5  # normalize to [-1, 1]
                images.append(img)
            except UnidentifiedImageError:
                print(f"Warning: Skipping file {filename} as it cannot be identified as an image.")
            except Exception as e:
                print(f"Error: {e} with file {filename}")
    return np.array(images)

cat = load_data('PetImages/Cat/', 64)
print("cat images uploaded")
dog = load_data('PetImages/Dog/', 64)
print("dog images uploaded")
data = np.concatenate([cat, dog])

cat images uploaded
dog images uploaded


In [7]:
print(data[0])

[[[ 0.60784316  0.3019608  -0.3019608 ]
  [ 0.6627451   0.35686275 -0.24705882]
  [ 0.73333335  0.4117647  -0.20784314]
  ...
  [ 0.9372549   0.6784314   0.07450981]
  [ 0.92156863  0.6313726  -0.00392157]
  [ 0.8901961   0.58431375 -0.03529412]]

 [[ 0.6156863   0.30980393 -0.29411766]
  [ 0.67058825  0.3647059  -0.23921569]
  [ 0.73333335  0.4117647  -0.20784314]
  ...
  [ 0.94509804  0.69411767  0.10588235]
  [ 0.9372549   0.654902    0.04313726]
  [ 0.92156863  0.6156863  -0.00392157]]

 [[ 0.6156863   0.30980393 -0.29411766]
  [ 0.67058825  0.3647059  -0.23921569]
  [ 0.73333335  0.4117647  -0.20784314]
  ...
  [ 0.92941177  0.69411767  0.12156863]
  [ 0.94509804  0.69411767  0.09019608]
  [ 0.9372549   0.6313726   0.01176471]]

 ...

 [[ 0.27058825 -0.00392157 -0.5372549 ]
  [ 0.3019608   0.02745098 -0.5058824 ]
  [ 0.34117648  0.05098039 -0.5058824 ]
  ...
  [-0.9764706  -0.96862745 -1.        ]
  [-0.9764706  -0.96862745 -1.        ]
  [-0.9843137  -0.9764706  -1.        ]]

 [

BUILD GENERATOR AND DISCRIMINATOR MODEL

In [8]:
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(128 * 8* 8, input_dim=latent_dim, activation='relu'))
    model.add(Reshape((8, 8, 128)))

    model.add(Conv2DTranspose(128, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(64, kernel_size=4, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2DTranspose(3, kernel_size=4, strides=2, padding="same"))
    return model
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(64, kernel_size=4, strides=2, input_shape=img_shape,padding="same"))
    model.add(LeakyReLU(alpha=0.01))

    model.add(Conv2D(128, kernel_size=4, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.01))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

In [9]:
def compile_gan(generator, discriminator):
    discriminator.trainable = False

    z = Input(shape=(latent_dim,))
    img = generator(z)
    validity = discriminator(img)

    combined = Model(z, validity)
    combined.compile(loss="binary_crossentropy", optimizer=Adam())
    return combined

latent_dim =100
generator = build_generator(latent_dim)
discriminator = build_discriminator((64, 64, 3))
gan = compile_gan(generator, discriminator)

discriminator.compile(loss="binary_crossentropy", 
                          optimizer=Adam(),
                          metrics=["accuracy"])

c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
from tensorflow.keras.models import save_model
def train_gan(gan, generator, discriminator, data, epochs, batch_size, latent_dim, save_interval = 100):
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    gan_model_path = "gan_models"
    if not os.path.exists(gan_model_path):
        os.makedirs(gan_model_path)
    for epoch in range(epochs):
        idx = np.random.randint(0, data.shape[0], batch_size)
        imgs = data[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)
        
        discriminator.trainable = True
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        discriminator.trainable = False
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch(noise, valid)

        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100*d_loss[1]}] [G loss: {g_loss}]")
        if epoch % save_interval == 0:
            generator.save(os.path.join(gan_model_path, f"generator_{epoch}.keras"))
            discriminator.save(os.path.join(gan_model_path, f"discriminator_{epoch}.keras"))
            gan.save(os.path.join(gan_model_path, f"gan_{epoch}.keras"))
train_gan(gan, generator, discriminator, data, epochs=10000, batch_size=64, latent_dim=latent_dim)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
0 [D loss: 0.5233521461486816 | D accuracy: 49.21875] [G loss: [array(0.56278044, dtype=float32), array(0.56278044, dtype=float32), array(0.421875, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step
1 [D loss: 0.4857025146484375 | D accuracy: 49.270832538604736] [G loss: [array(0.5203454, dtype=float32), array(0.5203454, dtype=float32), array(0.44791666, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
2 [D loss: 0.469914048910141 | D accuracy: 49.38616156578064] [G loss: [array(0.49370623, dtype=float32), array(0.49370623, dtype=float32), array(0.4609375, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
3 [D loss: 0.4530823230743408 | D accuracy: 49.47916567325592] [G loss: [array(0.4672509, dtype=float32), array(0.4672509, dtype=float32), array(0.46875, dtype=float32)]]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
4 [D loss: 0.4318947196006775 | D accuracy: 53.716856241226196] [G loss: [array(0.43893734, dtype=float32), array(0.4389

: 